<a href="https://colab.research.google.com/github/Mohammed-Taasir/CSE508_Winter2023_A2_14/blob/main/A2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/IR Assignment-1/CSE508_Winter2023_Dataset.zip' '/content/'
!unzip 'CSE508_Winter2023_Dataset.zip' &> /dev/null
!rm 'CSE508_Winter2023_Dataset.zip'

In [3]:
path = '/content/CSE508_Winter2023_Dataset/'

In [4]:
import glob
import pandas as pd
import numpy as np
import string
import os
import re
import pickle
import nltk
import random
import math
import heapq
import operator
from collections import Counter
from tqdm import tqdm

In [5]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import PlaintextCorpusReader 
from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Defining Helper Functions

### Read from files

In [7]:
  def getListOfFiles(directory):

    # Parameters: directory: type(string)        
    # returns: list of all files in directory with the full path of file
    
    list_of_files = []

    for i in range(len(filenames)):
      fi = "/content/CSE508_Winter2023_Dataset/"+filenames[i]
      list_of_files.append(fi)
    
    return list_of_files

### Preprocessing Functions

In [8]:
def lowercase(data):

    # Parameters: data: type(string)
    # returns: lowercase of data   
     
    return data.lower()

In [9]:
def perform_word_tokenize(corpus):
  
    # Parameters:corpus: type(string)   
    # returns word-level tokenization of corpus

    return word_tokenize(corpus)

In [10]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
  
    # Parameters: tokens: type(list)
    #             stopwords_set: type(set)
    # returns: tokens without stopwords

    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set] 
    return tokens_sans_stopwords

In [11]:
def remove_punctuation_from_tokens(tokens):

    # Parameters: tokens: type(list)
    # returns: tokens without punctuation

    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    return tokens_sans_punctuation

In [12]:
def remove_blank_space_tokens(tokens):
    
    #Parameters: tokens: type(list)
    #returns: tokens without blank tokens

    tokens_sans_blank_space = [x for x in tokens if x!='']  
    return tokens_sans_blank_space

In [13]:
def preprocess(corpus, stopwords_set):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    #print(len(lowercase_corpus))
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    #print(len(word_tokens))
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    #print(len(word_tokens_sans_stopwords))
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    #print(len(word_tokens_sans_punctuation))
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
    #print(len(word_tokens_sans_blank_tokens))
    
    # Stem tokens
    #word_tokens_final = stemming(word_tokens_sans_blank_tokens)
    
    return word_tokens_sans_blank_tokens

In [14]:
def preprocess_documents(list_of_files, stopwords_set):
    '''
    Parameters:
        list_of_files: type(list)
        stopwords_set: type(set)
    
    returns: list of tokens obtained by preprocessing documents in all classes
    '''
    preprocessed_list_of_docs_tokens = []
    for doc_path in list_of_files:
        file = open(doc_path, 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set)
        preprocessed_list_of_docs_tokens.append(doc_tokens)
    
    pi_file = open('Q1_tf_idf.pkl', 'wb')
    pickle.dump(preprocessed_list_of_docs_tokens, pi_file)
    pi_file.close()
    
    return preprocessed_list_of_docs_tokens

In [15]:
def create_file_dictionary(list_of_files):
    
    # Paramteres: list_of_files: type(string)
    # returns: file_dictionary with integer key and path_of_file as value
  
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

# Q1. Data Preprocessing

## (i) Relevant Text Extraction

In [16]:
corpus_root='/content/CSE508_Winter2023_Dataset/'
corpus=PlaintextCorpusReader(corpus_root,'.*')

In [17]:
# Printing names of all files

filenames=corpus.fileids()
# print(filenames)

In [18]:
print(type(filenames))
print(len(filenames))

<class 'list'>
1400


## First 5 Documents

In [19]:
# Printing contents of first 5 files before extracting

for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------
<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the des

## Extracting content between TITLE and TEXT tag from all documents.

In [20]:
for i in range(len(filenames)):
    outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
    doc = outfile.read()

    # initializing string
    test_str = doc

    # initializing tags
    tag1 = "TITLE"
    tag2 = "TEXT"

    # regex to extract required strings
    reg_str1 = "<"+tag1+">(.*?)</"+tag1+">"
    res1 = re.findall(reg_str1, test_str, re.DOTALL)

    reg_str2 = "<"+tag2+">(.*?)</"+tag2+">"
    res2 = re.findall(reg_str2, test_str, re.DOTALL)

    #Combining contents of TITLE and TEXT
    res = res1+res2
    
    s = res

    # using list comprehension
    listToStr = ' '.join([str(elem) for elem in s])

    writeFile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'w')
    L = listToStr

    writeFile.write(L)
    writeFile.close()

    outfile.close()

## First 5 Documents after extracting contents between TITLE and TEXT tag.

In [21]:
for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-----------------------------------------------

# (ii) Preprocessing

## 1. Lowercase the text

In [22]:
newfiles=[]
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=lowercase(filedata)
  newfiles.append(filedata)
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-------------------------------------------------

## 2. Perform tokenization

In [23]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=perform_word_tokenize(filedata)
  newfiles[i] = filedata
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'of', 'the', 'aerodynamics', 'of', 'a', 'wing', 'in', 'a', 'slipstream', '.', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'was', 'made', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angles', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratios', '.', 'the', 'results', 'were', 'intended', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatments', 'of', 'this', 'problem', '.', 'the', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'with', 'supporting', 'evidence', ',', 'showed', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produced', 'by', 'the', 'slipstream', 'was', 'due', 'to', 'a', '/destalling/', 'or', 'boundary-layer-control'

In [24]:
stopwords_set = set(stopwords.words('english'))

## 3. Remove stopwords

In [25]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_stopwords = remove_stopwords_from_tokens(filedata, stopwords_set)
  newfiles[i] = tokens_sans_stopwords
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '.', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '.', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '.', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'supporting', 'evidence', ',', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', '/destalling/', 'boundary-layer-control', 'effect', '.', 'integrated', 'remaining', 'lift', 'increment', ',', 'subtracting', 'destalling', 'lift', ',', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '.', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '.']
--

## 4. Remove punctuations

In [26]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_punctuation = remove_punctuation_from_tokens(filedata)
  newfiles[i] = tokens_sans_punctuation
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '', 'comparative', 'span', 'loading', 'curves', '', 'together', 'supporting', 'evidence', '', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', '', 'integrated', 'remaining', 'lift', 'increment', '', 'subtracting', 'destalling', 'lift', '', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '']
----------------

## 5. Remove blank space tokens

In [27]:
lent = 0
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_blank_space = remove_blank_space_tokens(filedata)
  newfiles[i] = tokens_sans_blank_space
  lent += len(newfiles[i])
print(len(newfiles))
print(lent)

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
127377
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', 'comparative', 'span', 'loading', 'curves', 'together', 'supporting', 'evidence', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', 'integrated', 'remaining', 'lift', 'increment', 'subtracting', 'destalling', 'lift', 'found', 'agree', 'well', 'potential', 'flow', 'theory', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment']
-------------------------------------------------

In [28]:
fileData = newfiles

# TF-IDF Matrix [25 points]

Defining helper functions

In [29]:
def create_file_dictionary(list_of_files):
    '''
    Paramteres:
        list_of_files: type(string)
    
    returns: file_dictionary with integer key and path_of_file as value
    '''
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

In [30]:
def create_2_way_mapping_of_term_and_doc(global_vocabulary):
    '''
    Parameter:
        global_vocabulary: type(list)
    
    returns: 2 dictionaries with key-value pair as term:docID and docID:term respectively
    '''
    term_vs_ID_dict = {}
    ID_vs_term_dict = {}
    
    for idx,term in enumerate(global_vocabulary):
        ID_vs_term_dict[idx] = term
        term_vs_ID_dict[term] = idx
    
    return term_vs_ID_dict, ID_vs_term_dict


'''
This code creates two dictionaries that will be used to map terms to their corresponding index in a matrix and vice versa.

The first dictionary, term_vs_ID_dict, maps each term in the global vocabulary to its corresponding index in the matrix. The keys in this dictionary are the terms themselves, and the values are the corresponding indices.

The second dictionary, ID_vs_term_dict, maps each index in the matrix to its corresponding term in the global vocabulary. The keys in this dictionary are the indices, and the values are the corresponding terms.

The for loop iterates over each term in the global_vocabulary list and assigns an index to it. The enumerate function is used to get both the index and the corresponding term in each iteration. The index is used as the key in the ID_vs_term_dict dictionary, and the term is used as the key in the term_vs_ID_dict dictionary.
'''

'\nThis code creates two dictionaries that will be used to map terms to their corresponding index in a matrix and vice versa.\n\nThe first dictionary, term_vs_ID_dict, maps each term in the global vocabulary to its corresponding index in the matrix. The keys in this dictionary are the terms themselves, and the values are the corresponding indices.\n\nThe second dictionary, ID_vs_term_dict, maps each index in the matrix to its corresponding term in the global vocabulary. The keys in this dictionary are the indices, and the values are the corresponding terms.\n\nThe for loop iterates over each term in the global_vocabulary list and assigns an index to it. The enumerate function is used to get both the index and the corresponding term in each iteration. The index is used as the key in the ID_vs_term_dict dictionary, and the term is used as the key in the term_vs_ID_dict dictionary.\n'

In [31]:
def get_total_words_in_doc(TF_IDF_doc_vs_term_matrix):  # this function calculates the total number of words in each document represented by a given TF-IDF matrix.
    return np.sum(TF_IDF_doc_vs_term_matrix, axis=1)    # axis=1 means row wise sum

In [32]:
def get_max_freq_term_in_doc(matrix):   # this function returns a 1D array containing the maximum value for each row of the input matrix. In other words, it returns an array of the maximum frequency terms in each document represented by the rows of the input matrix.
    return np.amax(matrix, axis=1)

In [33]:
def calculate_IDF(inverted_index, total_documents):   # this function returns the IDF dictionary, which contains the IDF values for each term in the inverted_index. The IDF values can be used in information retrieval tasks to weight the importance of each term in a document or query.
    IDF = {}
    for term in inverted_index:
        IDF[term] = np.log10(total_documents/(1 + len(inverted_index[term])))
    return IDF

In [34]:
def Binary_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):    # The function uses NumPy's "where" function to set values in the matrix to either 0 or 1. The condition in the "where" function is (TF_IDF_doc_vs_term_matrix <= 0), which checks whether the TF-IDF weight in each cell is less than or equal to 0. If it is, the value in that cell is set to 0; otherwise, it is set to 1.
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: binary weighting scheme
    '''
    return np.where((TF_IDF_doc_vs_term_matrix <= 0),0,1).astype('float')

In [35]:
def Raw_count_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):   # The function simply returns the input matrix, which means that the raw count weighting scheme is not applied to the matrix. Therefore, the output matrix will be the same as the input matrix, containing the TF-IDF weights of each term in each document. In other words, the function does not modify the input matrix and returns it as is.
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: raw count weighing scheme
    '''
    return TF_IDF_doc_vs_term_matrix

In [36]:
def Term_frequency_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix, total_words_in_doc):    # The function first transposes the input matrix using TF_IDF_doc_vs_term_matrix.T, so that the rows correspond to each document and the columns correspond to each term. It then divides each element in each row by the corresponding value in total_words_in_doc to obtain the TF value for each term in each document. The resulting matrix is then transposed back to the original shape, with rows corresponding to each term and columns corresponding to each document. Finally, the function returns the resulting term frequency weighting scheme as a numpy array.
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
        total_words_in_doc: type(np.array)
    
    returns: term frequency weighing scheme
    '''
    return (TF_IDF_doc_vs_term_matrix.T / total_words_in_doc).T

In [37]:
def Log_Normalization_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):   # Log normalization is a method used to transform the TF values in a TF-IDF matrix to reduce the impact of very frequent terms and increase the impact of rare terms. This helps to address the issue of common words having a high impact on the similarity calculation even though they may not carry much meaning. By applying a logarithmic transformation to the TF values, the values for very frequent terms will be reduced, while the values for rare terms will be increased. This is because the logarithmic function is an increasing function that grows more slowly as the input value increases. As a result, the impact of the most frequent terms is scaled down, while the impact of the less frequent terms is scaled up.
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: log normalization weighing scheme
    '''
    return np.log10(1+TF_IDF_doc_vs_term_matrix)

In [76]:
def Double_Normalization_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix, max_freq_term_in_doc):    # The function first transposes the input matrix TF_IDF_doc_vs_term_matrix and divides each row by the corresponding maximum frequency of any term in that document. This is done to normalize the values in the matrix. After normalization, the function applies the double normalization weighing scheme by multiplying the resulting matrix by a constant of 0.5 and adding 0.5 to it. This scaling of values between 0 and 1 is used to represent the relative importance of each term in each document.
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: double normalization weighing scheme
    '''
    matrix = (TF_IDF_doc_vs_term_matrix.T / max_freq_term_in_doc).T
    value = 0.5 + (0.5*matrix)

    return value

In [77]:
def compute_variants(matrix, total_terms, max_freq, size_of_global_vocabulary, IDF, ID_vs_term_dict, isQuery):
    '''
    returns: variants of TF-IDF
    '''
    Binary = Binary_weighing_TF_IDF(matrix)
    Raw_count = Raw_count_weighing_TF_IDF(matrix)
    Term_frequency = Term_frequency_weighing_TF_IDF(matrix, total_terms)
    Log_Normalization = Log_Normalization_weighing_TF_IDF(matrix)
    Double_Normalization = Double_Normalization_weighing_TF_IDF(matrix, max_freq)
    
    if(isQuery):
        for i in tqdm(range(size_of_global_vocabulary)):
            IDF_factor = IDF[ID_vs_term_dict[i]]
            Binary[i] *= IDF_factor
            Raw_count[i] *= IDF_factor
            Term_frequency[i] *= IDF_factor
            Log_Normalization[i] *= IDF_factor
            Double_Normalization[i] *= IDF_factor
    else:
        for i in tqdm(range(size_of_global_vocabulary)):
            IDF_factor = IDF[ID_vs_term_dict[i]]
            Binary[:,i] *= IDF_factor
            Raw_count[:,i] *= IDF_factor
            Term_frequency[:,i] *= IDF_factor
            Log_Normalization[:, i] *= IDF_factor
            Double_Normalization[:, i] *= IDF_factor
    
    return Binary, Raw_count, Term_frequency, Log_Normalization, Double_Normalization

In [60]:
def topN(N, TF_IDF, query_of_same_variant, file_dictionary):    # Here's a breakdown of what it means: relevant_docs is a variable that will store the output of the code. list() is a Python function that creates a new list object. sorted() is another Python function that sorts the items in an iterable object (such as a dictionary) and returns a new list. tf_idf_score.items() is a Python dictionary method that returns a view object that contains the key-value pairs of the dictionary. key=operator.itemgetter(1) specifies that the sort order should be based on the value of the dictionary items (the second element in each key-value pair). reverse=True specifies that the sort order should be in descending order (i.e., highest to lowest). [:N] is Python syntax for slicing a list. It specifies that only the first N elements of the sorted list should be included in the output. So, in summary, this line of code takes a dictionary called tf_idf_score that contains key-value pairs, where the keys are document IDs and the values are TF-IDF scores, and sorts the dictionary by the values (in descending order) and returns the top N documents (specified by the N variable) with the highest TF-IDF scores in a new list called relevant_docs.
    '''
    evaluates and displays top N relevant documents based on score
    '''
    tf_idf_score = np.dot(TF_IDF, query_of_same_variant)
    tf_idf_score = {file_dictionary[i]:tf_idf_score[i] for i in range(len(tf_idf_score))}
    
    relevant_docs = list(sorted(tf_idf_score.items(), key=operator.itemgetter(1),reverse=True))[:N]
    for docs in relevant_docs:
        print('Score: {}  Document: {}'.format(docs[1], docs[0]))
    print()

MAIN

In [81]:
def main():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset')
    
    # create dictionary of file with docID (integer) as key and full_path of file as value
    file_dictionary = create_file_dictionary(list_of_files)
    total_documents = len(file_dictionary)
    
    preprocessed_list_of_docs_tokens = preprocess_documents(list_of_files, stopwords_set)
    # pi_file = open('Q1_tf_idf.pkl', 'rb')
    # preprocessed_list_of_docs_tokens = pickle.load(pi_file)
    # pi_file.close()
    
    # Global list of terms
    global_list_of_terms = []
    for doc in preprocessed_list_of_docs_tokens:
        global_list_of_terms.extend(doc)
    
    # finding all distinct terms across all documents
    global_vocabulary = list(set(global_list_of_terms))
    size_of_global_vocabulary = len(global_vocabulary)
    
    # term vs docID 2 way mapping
    term_vs_ID_dict, ID_vs_term_dict = create_2_way_mapping_of_term_and_doc(global_vocabulary)
    
    # term-doc matrix (for storing frequency of each word of global_vocabulary in doc)
    inverted_index = {}
    size_of_TF_IDF_matrix = (total_documents, size_of_global_vocabulary)                      # Create a matrix of size no. of documents x vocab size.    (RUBRIC)
    TF_IDF_doc_vs_term_matrix = np.zeros(size_of_TF_IDF_matrix, dtype=float)
    
    for i in tqdm(range(len(preprocessed_list_of_docs_tokens))):
        for term in preprocessed_list_of_docs_tokens[i]:
            if(term in inverted_index):
                if(inverted_index[term][-1]!=i):
                    inverted_index[term].append(i)
            else:
                inverted_index[term] = [i]
            TF_IDF_doc_vs_term_matrix[i][term_vs_ID_dict[term]] += 1
    
    total_words_in_doc = get_total_words_in_doc(TF_IDF_doc_vs_term_matrix)    # this function calculates the total number of words in each document represented by a given TF-IDF matrix.
    
    max_freq_term_in_doc = get_max_freq_term_in_doc(TF_IDF_doc_vs_term_matrix)    # this function returns a 1D array containing the maximum value for each row of the input matrix. In other words, it returns an array of the maximum frequency terms in each document represented by the rows of the input matrix.
    
    # Calculate IDF
    IDF = calculate_IDF(inverted_index, total_documents)
    
    # Fill in the tf-idf values for each term in the vocabulary in the matrix.    (RUBRIC)
    Binary_tf_idf, Raw_count_tf_idf, Term_frequency_tf_idf, Log_Normalization_tf_idf, Double_Normalization_tf_idf = compute_variants(TF_IDF_doc_vs_term_matrix, total_words_in_doc, max_freq_term_in_doc, size_of_global_vocabulary, IDF, ID_vs_term_dict, False)
    
    query = input("Input query: ")
    sanitized_query = preprocess(query, stopwords_set)
    print("Sanitized query: ", sanitized_query)

    query_frequency = {}
    for query_token in sanitized_query:
        if query_token in query_frequency:
            query_frequency[query_token]+=1
        else:
            query_frequency[query_token]=1
    
    # create query frequency vector
    query_frequency_vector = np.zeros((size_of_global_vocabulary,1))                        # Construct the query vector of size vocab.   (RUBRIC)
    for token in query_frequency.keys():
        if token in global_vocabulary:
            query_frequency_vector[term_vs_ID_dict[token]] = query_frequency[token]
    
    
    max_freq_token = query_frequency[max(query_frequency, key=query_frequency.get)]
    # print(max(query_frequency, key=query_frequency.get), max_freq_token, "just to test")

    # Compute the TF-IDF score for the query using the TF-IDF matrix. Report the top 5 relevant documents based on the score.   (RUBRIC)
    query_binary, query_raw_count, query_term_frequency, query_log_normalization, query_double_normalization = compute_variants(query_frequency_vector, len(sanitized_query), max_freq_token, size_of_global_vocabulary, IDF, ID_vs_term_dict, True)
    

    # Use all 5 weighting schemes for term frequency calculation and report the TF-IDF score and results for each scheme separately.    (RUBRIC)
    print('\n\nBinary Scheme: Top 5 relevant documents are:')
    topN(5, Binary_tf_idf, query_binary, file_dictionary)
    print(Binary_tf_idf, "\n\n")
    
    print('Raw Count Scheme: Top 5 relevant documents are:')
    topN(5, Raw_count_tf_idf, query_raw_count, file_dictionary)
    print(Raw_count_tf_idf, "\n\n")
    
    print('Term Frequency Scheme: Top 5 relevant documents are:')
    topN(5, Term_frequency_tf_idf, query_term_frequency, file_dictionary)
    print(Term_frequency_tf_idf, "\n\n")
    
    print('Log Normalization Scheme: Top 5 relevant documents are:')
    topN(5, Log_Normalization_tf_idf, query_log_normalization, file_dictionary)
    print(Log_Normalization_tf_idf, "\n\n")
    
    print('Double_Normalization Scheme: Top 5 relevant documents are:')
    topN(5, Double_Normalization_tf_idf, query_double_normalization, file_dictionary)
    print(Double_Normalization_tf_idf, "\n\n")

In [83]:
query = "cosmonautics is currently very much to the forefront in the news"    # cranfield718

In [84]:
if __name__ == "__main__":
    main()

100%|██████████| 8996/8996 [00:00<00:00, 14186.09it/s]


Input query: cosmonautics is currently very much to the forefront in the news
Sanitized query:  ['cosmonautics', 'currently', 'much', 'forefront', 'news']
cosmonautics 1 YOUR ERROR COMES HERE


100%|██████████| 8996/8996 [00:00<00:00, 32962.00it/s]




Binary Scheme: Top 5 relevant documents are:
Score: [30.1560637]  Document: /content/CSE508_Winter2023_Dataset/cranfield0718
Score: [4.12977459]  Document: /content/CSE508_Winter2023_Dataset/cranfield0092
Score: [4.12977459]  Document: /content/CSE508_Winter2023_Dataset/cranfield0115
Score: [4.12977459]  Document: /content/CSE508_Winter2023_Dataset/cranfield0367
Score: [4.12977459]  Document: /content/CSE508_Winter2023_Dataset/cranfield0506

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 


Raw Count Scheme: Top 5 relevant documents are:
Score: [31.89860423]  Document: /content/CSE508_Winter2023_Dataset/cranfield0718
Score: [5.22762161]  Document: /content/CSE508_Winter2023_Dataset/cranfield0114
Score: [4.12977459]  Document: /content/CSE508_Winter2023_Dataset/cranfield0092
Score: [4.12977459]  Document: /content/CSE508_Winter2023_Dataset/cranfield0115
Score: [4.12977459]  Doc

#Jaccard Coefficient [15 marks]

Union function

In [85]:
def union(l1,l2):
    # union_list = (set(l1) | set(l2))
    l1 = set(l1)
    l2 = set(l2)
    return l1.union(l2)

Intersection function

In [86]:
def intersection(l1,l2):
    # intersection_list = (set(l1) & set(l2))
    # return intersection_list
    l1 = set(l1)
    l2 = set(l2)
    return l1.intersection(l2)

Calculate Jaccard coefficient

In [87]:
def jaccardCoefficient(query, list_of_files):
    for i,filePath in enumerate (list_of_files):
        file = open(filePath, encoding="utf8", errors = "ignore")
        read = file.read()    
        file.close()
    
        sanitized_query = preprocess(read, stopwords_set)                    
        # calculate jaccard coefficient value based on formula that is intersection of document and query divided by union of 
        # document and query
        jaccard_coefficient[filePath] = len(intersection(sanitized_query, query))/len(union(sanitized_query, query))

In [88]:
len(stopwords_set)

179

Finding top 10 relevant documents

In [90]:
query = "cosmonautics is currently very much to the forefront in the news"

In [89]:
# Get List of Files in Dataset
list_of_files = getListOfFiles("/content/CSE508_Winter2023_Dataset/")
#print(list_of_files)
jaccard_coefficient = {}        
sentence_query = input("Enter the query: ")
query = preprocess(sentence_query, stopwords_set)      # Query Processing
jaccardCoefficient(query, list_of_files)
# it counts the elements in the dictionary and prints the 5 most common documents based on it.
#k = Counter(dict(jaccard_coefficient)).most_common(5) 
print(" Top 10 relevant documents based on Jaccard Coefficient ")  
for i in Counter(dict(jaccard_coefficient)).most_common(10) :
    print("{} --> {}".format(i[1],i[0]))

Enter the query: cosmonautics is currently very much to the forefront in the news
 Top 10 relevant documents based on Jaccard Coefficient 
0.07246376811594203 --> /content/CSE508_Winter2023_Dataset/cranfield0718
0.02857142857142857 --> /content/CSE508_Winter2023_Dataset/cranfield0909
0.027777777777777776 --> /content/CSE508_Winter2023_Dataset/cranfield1111
0.023809523809523808 --> /content/CSE508_Winter2023_Dataset/cranfield1369
0.021739130434782608 --> /content/CSE508_Winter2023_Dataset/cranfield0840
0.02 --> /content/CSE508_Winter2023_Dataset/cranfield1033
0.019230769230769232 --> /content/CSE508_Winter2023_Dataset/cranfield1396
0.018518518518518517 --> /content/CSE508_Winter2023_Dataset/cranfield0383
0.01818181818181818 --> /content/CSE508_Winter2023_Dataset/cranfield0866
0.01818181818181818 --> /content/CSE508_Winter2023_Dataset/cranfield1052


In [ ]:
# queries -> jet propulsion, supersonic wind tunnel, experimental study of a wing in a propeller slipstream, differential equations for a heated plate with large temperature gradient <cranfield13>, there are in supersonic aerodynamics many situations of practical interest wherein streams of different velocities and, in general <cranfield11>